In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import udf, to_timestamp
import happybase
import datetime
import jieba
import spacy
from spacy.lang.id import Indonesian
from spacy.lang.en import English
from nltk.corpus import stopwords
from pyspark.sql.types import ArrayType, StringType
import re
from googletrans import Translator
import contractions
from nltk.stem import WordNetLemmatizer
import demoji
import string
import datetime, time

In [2]:
# creating the session
spark = SparkSession.builder \
    .config("spark.mongodb.read.connection.uri", "mongodb://127.0.0.1/data_engineering.preprocessed.coll") \
    .config("spark.mongodb.write.connection.uri", "mongodb://127.0.0.1/data_engineering.preprocessed.coll") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.1.1") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/05/01 15:05:49 WARN Utils: Your hostname, LAPTOP-KHLEE resolves to a loopback address: 127.0.1.1; using 172.24.76.154 instead (on interface eth0)
23/05/01 15:05:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hduser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hduser/.ivy2/cache
The jars for the packages stored in: /home/hduser/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-59eae472-99f0-4f91-87ed-89c106b5e195;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.1.1 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
:: resolution report :: resolve 3807ms :: artifacts dl 27ms
	:: modules in use:
	org.mongodb#bson;4.8.2 from central in [default]
	org.mongodb#bson-record-codec;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-core;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-sync;4.8.2 from central in [default]
	org.mongodb.spark#mongo-spark-connector

In [3]:
connection = happybase.Connection(port=9090)
hbase_table = connection.table('redditMalaysia')

df = pd.DataFrame()                    

edited, spoiler, url, name, title, created_utc, num_comments, score = [], [], [], [], [], [], [], []

# function to convert string to boolean
def str_to_bool(s: int) -> bool:
    return (lambda x: True if x == "True" else False) (s)

# function to convert integer utc to timestamp
def convert_timestamp(s: int) -> datetime:
    utc_datetime = datetime.datetime.utcfromtimestamp(s)
    timestamp_str = utc_datetime.strftime('%Y-%m-%d %H:%M:%S')
    return timestamp_str

for key, data in hbase_table.scan():
    edited.append(str_to_bool(data[b'cf1:edited'].decode('utf-8')))
    spoiler.append(str_to_bool(data[b'cf1:spoiler'].decode('utf-8')))
    url.append(data[b'cf2:url'].decode('utf-8'))
    name.append(data[b'cf2:name'].decode('utf-8'))
    title.append(data[b'cf2:sr_title'].decode('utf-8'))
    created_utc.append(convert_timestamp(int(float(data[b'cf3:created_utc'].decode('utf-8')))))
    num_comments.append(int(data[b'cf3:num_comments'].decode('utf-8')))
    score.append(int(data[b'cf3:score'].decode('utf-8')))


# add list to dataframe
df['edited'], df['spoiler'], df['url'], df['name'], df['title'], df['created_utc'], df['num_comments'], df['score'] = edited, spoiler, url, name, title, created_utc, num_comments, score
del edited, spoiler, url, name, title, created_utc, num_comments, score

df.head()

,edited,spoiler,url,name,title,created_utc,num_comments,score
0,False,False,https://i.redd.it/12z4zp8cbh691.jpg,t3_vfjzol,昨天有谁在追印尼公开赛李梓嘉挑战安赛龙？ 最后时刻李梓嘉一下子挽救4个赛点，并以21比20实...,2022-06-19 01:16:02,1,1
1,False,False,https://www.nst.com.my/news/nation/2023/04/903...,t3_132etb8,PM Anwar to Kedahans: Support unity government...,2023-04-29 00:45:54,10,69
2,False,False,https://twitter.com/NewsBFM/status/16512435764...,t3_130brnu,The Australian government's declassified repor...,2023-04-27 06:10:49,9,19
3,False,False,https://www.reddit.com/gallery/zfnjwn,t3_zfnjwn,Naoshima Island 直島 - 濑户内海艺术祭Setouchi Trinale日本...,2022-12-08 03:28:42,0,3
4,False,False,https://www.reddit.com/r/malaysia_zh/comments/...,t3_vykpq8,马来西亚2022年雇佣（修正）法令将在今年9月1日正式生效后，雇员可向雇主申请灵活上班,2022-07-14 02:24:17,1,4


In [4]:
# convert to spark dataframe (due to spark 3.3.2 , we need to downgrade pandas to 1.5.3)
spark_df = spark.createDataFrame(df)

/home/hduser/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/hduser/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [5]:
# show table
spark_df.show(5)

+------+-------+--------------------+----------+-------------------------------------+-------------------+------------+-----+
|edited|spoiler|                 url|      name|                                title|        created_utc|num_comments|score|
+------+-------+--------------------+----------+-------------------------------------+-------------------+------------+-----+
| false|  false|https://i.redd.it...| t3_vfjzol|昨天有谁在追印尼公开赛李梓嘉挑战安...|2022-06-19 01:16:02|           1|    1|
| false|  false|https://www.nst.c...|t3_132etb8|                 PM Anwar to Kedah...|2023-04-29 00:45:54|          10|   69|
| false|  false|https://twitter.c...|t3_130brnu|                 The Australian go...|2023-04-27 06:10:49|           9|   19|
| false|  false|https://www.reddi...| t3_zfnjwn|                Naoshima Island 直...|2022-12-08 03:28:42|           0|    3|
| false|  false|https://www.reddi...| t3_vykpq8|    马来西亚2022年雇佣（修正）法令...|2022-07-14 02:24:17|           1|    4|
+------+-------+-------

In [6]:
# show schema
spark_df.printSchema()

root
 |-- edited: boolean (nullable = true)
 |-- spoiler: boolean (nullable = true)
 |-- url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- num_comments: long (nullable = true)
 |-- score: long (nullable = true)



In [7]:
# convert datetime string to timestamp
spark_df = spark_df.select(
    'edited', 'spoiler', 'url', 'name', 'title', 'num_comments', 'score',
    to_timestamp(spark_df.created_utc, 'yyyy-MM-dd HH:mm:ss').alias('created_utc')
)

In [8]:
# get the number of rows
spark_df.count()

366

In [9]:
# get data of a specific date
dates = ("2023-04-27 00:00:00",  "2023-05-01 00:00:00")

timestamps = (
    time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple())
    for s in dates)

q1 = "CAST(created_utc AS INT) BETWEEN {0} AND {1}".format(*timestamps)
spark_df = spark_df.where(q1)

In [10]:
# get the number of rows
spark_df.count()

134

In [11]:
# get the title column
spark_df.select('title').show(n=10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                                                                                                                                                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|PM Anwar to Kedahans: Support unity government for our policy, not because of meals                                                                                                             

### Text Preprocessing

In [12]:
def preprocess_text(text):
    translator = Translator()
    lemmatizer = WordNetLemmatizer()

    # convert to lowercase
    text = text.lower()

    # contraction handler
    expanded_words = ' '.join([contractions.fix(word) for word in text.split()])
    text = expanded_words
    del expanded_words

    # remove numbers
    text = re.sub(r'\d+', '', text)
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # lemmatize and tokenize text
    tokens = jieba.lcut(text)

    # remove ‘ ’ from tokens
    tokens = [token for token in tokens if token not in ['‘', '’']]

    # remove whitespaces
    tokens = [token for token in tokens if token.strip()]

    # find all emojis in text
    emojis = demoji.findall(text)
    
    # if chinese then skip lemmatization
    for token in tokens:
        # convert emoji to text form, skip preprocessing if emoji
        if token in emojis:
            tokens[tokens.index(token)] = emojis[token]
            continue

        token_lang = translator.detect(token)
        print(token, token_lang.lang)

        if token_lang.lang == 'zh-CN':
            # skip lemmatization as it is not needed for chinese
            # translate chinese to english
            tokens[tokens.index(token)] = translator.translate(token, dest='en', src='zh-CN').text
        elif token_lang.lang == 'en':
            # lemmatize text
            tokens[tokens.index(token)] = lemmatizer.lemmatize(token)

        elif token_lang.lang == 'ms':
            # translate indonesian to english
            tokens[tokens.index(token)] = translator.translate(token, dest='en', src='ms').text
    
    # remove stopwords
    return  [token for token in tokens if token not in stopwords.words('english') and token not in stopwords.words('indonesian') and token not in stopwords.words('chinese') and token]

udf_preprocess_text = udf(preprocess_text, ArrayType(StringType()))

In [14]:
spark_df = spark_df.withColumn('tokens', udf_preprocess_text('title'))

In [15]:
# before and after preprocessed column
spark_df.select('title', 'tokens').show(n=20, truncate=False)

Building prefix dict from the default dictionary ...                (0 + 1) / 1]
Loading model from cache /tmp/jieba.cache
Loading model cost 1.550 seconds.
Prefix dict has been built successfully.
pm en
anwar en
to en
kedahans ms
support en
unity en
government en
for en
our en
policy en
not en
because en
of en
meals en
the en
australian en
governments en
declassified en
report en
on en
the en
plane en
crash en
says en
the en
main en
because en
was en
the en
overloading en
of en
the en
plane en
it en
also en
claimed en
that en
sabah en
air en
had en
illegal en
and en
poor en
operations en
and en
said en
pilot en
gandhi hi
nathan en
had en
substandard en
skills en
in en
flying en
the en
aircraft en
akka om
nasi en
lemak en
proprietor en
must en
learn en
from en
mistake en
of en
youtube en
cooking en
sensation en
sugu sw
pavithra te
help en
malaysian en
telco en
lock en
padi en
milling en
in en
sarawak en
circa en
s en
penang en
south en
island en
reclamation en
will en
not en
affect en


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                                                                                                                                                                                                                                                     |tokens                                                                                                                                                                                                                 |
+-----------------------------

In [ ]:
# get the number of rows
spark_df.count()

### Dump the preprocessed data into mongodb database

In [17]:
# save the entire pyspark dataframe into the database
spark_df.drop("edited", "spoilers", "name", "num_comments").write.format("mongodb").mode("overwrite").option("maxRecordsPerFile", 2).partitionBy("tokens").save()

23/05/01 15:28:52 WARN CaseInsensitiveStringMap: Converting duplicated key maxrecordsperfile into CaseInsensitiveStringMap.
Building prefix dict from the default dictionary ...                (0 + 8) / 8]
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 5.494 seconds.
Prefix dict has been built successfully.
Loading model